In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [ ]:
from sysgym import EnvParamsDict

import torch
from autorocks.envs.synthetic.funcs import levy
from pathlib import Path
from autorocks.optimizer.bograph.bobn import BoBn

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}


In [4]:
import pandas as pd
import gc
import time


def clear_memory():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            if "cuda" in str(obj.device):
                obj.detach().to("cpu")
                del obj
    # torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.reset_peak_memory_stats("cuda")


class MemoryExp:

    def __init__(self, dim: int):
        assert (torch.cuda.is_available())
        self._levy_struct = levy.make_struct(dim)
        self._space = levy.make_levy_space(dim)
        self._problem = levy.LevyND(levy.LevyCfg(dim), Path("/tmp/"))
        self._bobn = BoBn(self._levy_struct, self._space, {"target"}, conservative_mode=True)
        self._dim = dim

    def run(self, observations: int, restart_id):
        ds = self.generate_dataset(observations)
        clear_memory()
        start_memory = torch.cuda.max_memory_allocated("cuda")
        start_time = time.time()
        try:
            self._bobn.opt(ds)
        except torch.cuda.OutOfMemoryError:
            return {"peak_memory": -99,
                    'runtime_seconds': time.time() - start_time,
                    'restart_id': restart_id,
                    'observations': observations}
        elapsed_time = time.time() - start_time
        total_memory_usage = torch.cuda.max_memory_allocated("cuda") - start_memory

        return {"peak_memory_bytes": total_memory_usage,
                'runtime_seconds': elapsed_time,
                'restart_id': restart_id,
                'observations': observations,
                'dim': self._dim}

    def generate_dataset(self, n: int) -> pd.DataFrame:
        res = []
        param_dict = EnvParamsDict(self._space)
        obs_x = self._bobn.generate_initial_points(n)
        for x in obs_x:
            param_dict.update(x)
            result_dict = self._problem.run(param_dict).as_flat_dict()
            result_dict.update(dict(param_dict))
            res.append(result_dict)

        return pd.DataFrame(res)

# bobn.opt(generate_dataset(1000))

In [8]:
EXP_DIM = [6, 25, 50, 100, 1000]
OBS = [10, 100, 500, 1000, 10000]
all_results = []
n_restarts = 3

for obs in OBS:
    for exp_dim in EXP_DIM:
        exp = MemoryExp(exp_dim)
        for restart_id in range(n_restarts):
            print(f"Performing experiment: {restart_id=}:{exp_dim=}:{obs=}")
            all_results.append(exp.run(obs, restart_id))

Performing experiment: restart_id=0:exp_dim=6:obs=10


/home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


Performing experiment: restart_id=1:exp_dim=6:obs=10


/home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


Performing experiment: restart_id=2:exp_dim=6:obs=10


...

Performing experiment: restart_id=2:exp_dim=100:obs=10000


/home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


Performing experiment: restart_id=0:exp_dim=1000:obs=10000


/home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


RuntimeError: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/torch/autograd/graph.py(121): pack_to_cpu
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/linear_operator/utils/cholesky.py(20): _psd_safe_cholesky
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/linear_operator/utils/cholesky.py(65): psd_safe_cholesky
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/linear_operator/operators/_linear_operator.py(494): _cholesky
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/linear_operator/utils/memoize.py(59): g
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/linear_operator/operators/_linear_operator.py(1244): cholesky
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/gpytorch/distributions/multivariate_normal.py(88): _unbroadcasted_scale_tril
  /home/salabed/anaconda3/envs/rocksdb/lib/python3.8/site-packages/torch/distri

In [10]:
res_df = pd.DataFrame(all_results)

res_df.to_csv("scalability_dim.csv")

In [9]:
print("hello")

hello


In [12]:
EXP_DIM = [6, 25, 50, 100, 1000]
OBS = [10, 100, 500, 1000, 10000]
n_restarts = 3

missing_col = []
for obs in OBS:
    for exp_dim in EXP_DIM:
        for restart_id in range(n_restarts):
            missing_col.append(exp_dim)

pd.DataFrame(missing_col)

,0
0,6
1,6
2,6
3,25
4,25
...,...
70,100
71,100
72,1000
73,1000


In [15]:
res_df['dim'] = pd.DataFrame(missing_col)


In [17]:
res_df.to_csv("scalability_dims.csv")
